### For simulated data, we run the same clustering and tree inference algorithm as MACHINA for better comparison. Their tree inference algorithm gets rid of clusters sometimes. In those instances, fix up the tsvs with the correct character index to character label so that the inferred trees appropriately correspond

In [1]:
import os

repo_dir = os.path.join(os.getcwd(), "../")
machina_sims_data_dir = os.path.join(repo_dir, 'data', 'machina_sims')

In [16]:
import fnmatch
from metient.util import data_extraction_util as dutil
import pandas as pd
import numpy as np
import csv

sites = ["m8", "m5"]
mig_types = ["M", "mS", "R", "S"]
final_cols = ['anatomical_site_index','anatomical_site_label', 'cluster_index', 'character_index','character_label', 'ref', 'var', 'var_read_prob', 'site_category']

for site in sites:
    for mig_type in mig_types:
        site_mig_data_dir = os.path.join(machina_sims_data_dir, site, mig_type)
        seeds = fnmatch.filter(os.listdir(site_mig_data_dir), 'reads_seed*.tsv')
        seeds = [s.replace(".tsv", "").replace("reads_seed", "") for s in seeds]
        for seed in seeds:
            cluster_fn = os.path.join(machina_sims_data_dir, f"{site}_clustered_input", f"cluster_{mig_type}_seed{seed}.txt")
            all_mut_trees_fn = os.path.join(machina_sims_data_dir, f"{site}_mut_trees", f"mut_trees_{mig_type}_seed{seed}.txt")
            ref_var_fn = os.path.join(site_mig_data_dir, f"reads_seed{seed}.tsv")
#             ref_var_fn = os.path.join(machina_sims_data_dir, f"{site}_clustered_input", f"cluster_{mig_type}_seed{seed}.tsv")

            idx_to_cluster_label = dutil.get_idx_to_cluster_label(cluster_fn, ignore_polytomies=True)
            data = dutil.get_adj_matrices_from_spruce_mutation_trees(all_mut_trees_fn, idx_to_cluster_label, is_sim_data=True)

            for tree_num, (adj_matrix, pruned_idx_to_clstr_label) in enumerate(data):
                # Use the pruned index to cluster label to build a map between each 
                # mutation and its cluster index assignment
                mut_name_to_cluster_idx = {}
                for clstr_idx in pruned_idx_to_clstr_label:
                    muts = pruned_idx_to_clstr_label[clstr_idx].split(";")
                    for mut in muts:
                        mut_name_to_cluster_idx[int(mut)] = int(clstr_idx)
                df = pd.read_csv(ref_var_fn, sep="\t", skiprows=3)
                df['var_read_prob'] = 0.5
                df['site_category'] = df.apply(lambda row: 'primary' if row['anatomical_site_label']=="P" else 'metastasis', axis=1)
                # take out any mutations not used in the adjacency matrix (i.e. no cluster assignment anymore)
                x = len(df)
                df['cluster_index'] = df.apply(lambda row: int(mut_name_to_cluster_idx[row['character_label']]) if row['character_label'] in mut_name_to_cluster_idx else np.nan, axis=1)
                df = df.dropna(subset=['cluster_index'])
                df = df[final_cols]
                
                #if site=='m8' and mig_type=='M' and seed=='19' and tree_num==0:
#                 if len(df) != x:
                print(site, mig_type, seed, tree_num)
                print(idx_to_cluster_label)
                print(pruned_idx_to_clstr_label)
                print(mut_name_to_cluster_idx)
                print(df['cluster_index'].unique())
                
                output_fn = os.path.join(machina_sims_data_dir, f"{site}_clustered_input_corrected", f"cluster_{mig_type}_seed{seed}_tree{tree_num}.tsv")
                with open(output_fn, mode='w', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file, delimiter='\t')
                    writer.writerow(df.columns)
                    for _, row in df.iterrows():
                        writer.writerow(row)
                

m8 S 0 0
OrderedDict([(0, '0'), (1, '2'), (2, '4'), (3, '11'), (4, '5;9;13;15'), (5, '6;8;10;16;20'), (6, '23'), (7, '12;14;31'), (8, '1;3;17;18;30;32'), (9, '34;38'), (10, '35;37;44;46;50'), (11, '51;55'), (12, '57'), (13, '61'), (14, '63'), (15, '19;21;49;58;65;66'), (16, '56;67;69;71'), (17, '72'), (18, '74'), (19, '59;76'), (20, '39;40;47;54;60;70;75;77'), (21, '24;29;33;41;45;52;79'), (22, '82')])
{0: '0', 1: '2', 2: '11', 3: '5;9;13;15', 4: '6;8;10;16;20', 5: '23', 6: '12;14;31', 7: '1;3;17;18;30;32', 8: '34;38', 9: '35;37;44;46;50', 10: '51;55', 11: '57', 12: '61', 13: '63', 14: '19;21;49;58;65;66', 15: '56;67;69;71', 16: '72', 17: '74', 18: '59;76', 19: '39;40;47;54;60;70;75;77', 20: '24;29;33;41;45;52;79', 21: '82'}
{0: 0, 2: 1, 11: 2, 5: 3, 9: 3, 13: 3, 15: 3, 6: 4, 8: 4, 10: 4, 16: 4, 20: 4, 23: 5, 12: 6, 14: 6, 31: 6, 1: 7, 3: 7, 17: 7, 18: 7, 30: 7, 32: 7, 34: 8, 38: 8, 35: 9, 37: 9, 44: 9, 46: 9, 50: 9, 51: 10, 55: 10, 57: 11, 61: 12, 63: 13, 19: 14, 21: 14, 49: 14, 58: 1

m8 S 0 7
OrderedDict([(0, '0'), (1, '2'), (2, '4'), (3, '11'), (4, '5;9;13;15'), (5, '6;8;10;16;20'), (6, '23'), (7, '12;14;31'), (8, '1;3;17;18;30;32'), (9, '34;38'), (10, '35;37;44;46;50'), (11, '51;55'), (12, '57'), (13, '61'), (14, '63'), (15, '19;21;49;58;65;66'), (16, '56;67;69;71'), (17, '72'), (18, '74'), (19, '59;76'), (20, '39;40;47;54;60;70;75;77'), (21, '24;29;33;41;45;52;79'), (22, '82')])
{0: '0', 1: '2', 2: '11', 3: '5;9;13;15', 4: '6;8;10;16;20', 5: '23', 6: '12;14;31', 7: '1;3;17;18;30;32', 8: '34;38', 9: '35;37;44;46;50', 10: '51;55', 11: '57', 12: '61', 13: '63', 14: '19;21;49;58;65;66', 15: '56;67;69;71', 16: '72', 17: '74', 18: '59;76', 19: '39;40;47;54;60;70;75;77', 20: '24;29;33;41;45;52;79', 21: '82'}
{0: 0, 2: 1, 11: 2, 5: 3, 9: 3, 13: 3, 15: 3, 6: 4, 8: 4, 10: 4, 16: 4, 20: 4, 23: 5, 12: 6, 14: 6, 31: 6, 1: 7, 3: 7, 17: 7, 18: 7, 30: 7, 32: 7, 34: 8, 38: 8, 35: 9, 37: 9, 44: 9, 46: 9, 50: 9, 51: 10, 55: 10, 57: 11, 61: 12, 63: 13, 19: 14, 21: 14, 49: 14, 58: 1

m8 S 0 16
OrderedDict([(0, '0'), (1, '2'), (2, '4'), (3, '11'), (4, '5;9;13;15'), (5, '6;8;10;16;20'), (6, '23'), (7, '12;14;31'), (8, '1;3;17;18;30;32'), (9, '34;38'), (10, '35;37;44;46;50'), (11, '51;55'), (12, '57'), (13, '61'), (14, '63'), (15, '19;21;49;58;65;66'), (16, '56;67;69;71'), (17, '72'), (18, '74'), (19, '59;76'), (20, '39;40;47;54;60;70;75;77'), (21, '24;29;33;41;45;52;79'), (22, '82')])
{0: '0', 1: '2', 2: '4', 3: '11', 4: '5;9;13;15', 5: '23', 6: '12;14;31', 7: '1;3;17;18;30;32', 8: '34;38', 9: '35;37;44;46;50', 10: '51;55', 11: '57', 12: '61', 13: '63', 14: '19;21;49;58;65;66', 15: '56;67;69;71', 16: '72', 17: '74', 18: '59;76', 19: '39;40;47;54;60;70;75;77', 20: '24;29;33;41;45;52;79', 21: '82'}
{0: 0, 2: 1, 4: 2, 11: 3, 5: 4, 9: 4, 13: 4, 15: 4, 23: 5, 12: 6, 14: 6, 31: 6, 1: 7, 3: 7, 17: 7, 18: 7, 30: 7, 32: 7, 34: 8, 38: 8, 35: 9, 37: 9, 44: 9, 46: 9, 50: 9, 51: 10, 55: 10, 57: 11, 61: 12, 63: 13, 19: 14, 21: 14, 49: 14, 58: 14, 65: 14, 66: 14, 56: 15, 67: 15, 69

m8 S 0 25
OrderedDict([(0, '0'), (1, '2'), (2, '4'), (3, '11'), (4, '5;9;13;15'), (5, '6;8;10;16;20'), (6, '23'), (7, '12;14;31'), (8, '1;3;17;18;30;32'), (9, '34;38'), (10, '35;37;44;46;50'), (11, '51;55'), (12, '57'), (13, '61'), (14, '63'), (15, '19;21;49;58;65;66'), (16, '56;67;69;71'), (17, '72'), (18, '74'), (19, '59;76'), (20, '39;40;47;54;60;70;75;77'), (21, '24;29;33;41;45;52;79'), (22, '82')])
{0: '0', 1: '2', 2: '4', 3: '11', 4: '5;9;13;15', 5: '23', 6: '12;14;31', 7: '1;3;17;18;30;32', 8: '34;38', 9: '35;37;44;46;50', 10: '51;55', 11: '57', 12: '61', 13: '63', 14: '19;21;49;58;65;66', 15: '56;67;69;71', 16: '72', 17: '74', 18: '59;76', 19: '39;40;47;54;60;70;75;77', 20: '24;29;33;41;45;52;79', 21: '82'}
{0: 0, 2: 1, 4: 2, 11: 3, 5: 4, 9: 4, 13: 4, 15: 4, 23: 5, 12: 6, 14: 6, 31: 6, 1: 7, 3: 7, 17: 7, 18: 7, 30: 7, 32: 7, 34: 8, 38: 8, 35: 9, 37: 9, 44: 9, 46: 9, 50: 9, 51: 10, 55: 10, 57: 11, 61: 12, 63: 13, 19: 14, 21: 14, 49: 14, 58: 14, 65: 14, 66: 14, 56: 15, 67: 15, 69

m8 S 0 34
OrderedDict([(0, '0'), (1, '2'), (2, '4'), (3, '11'), (4, '5;9;13;15'), (5, '6;8;10;16;20'), (6, '23'), (7, '12;14;31'), (8, '1;3;17;18;30;32'), (9, '34;38'), (10, '35;37;44;46;50'), (11, '51;55'), (12, '57'), (13, '61'), (14, '63'), (15, '19;21;49;58;65;66'), (16, '56;67;69;71'), (17, '72'), (18, '74'), (19, '59;76'), (20, '39;40;47;54;60;70;75;77'), (21, '24;29;33;41;45;52;79'), (22, '82')])
{0: '0', 1: '2', 2: '4', 3: '11', 4: '5;9;13;15', 5: '23', 6: '12;14;31', 7: '1;3;17;18;30;32', 8: '34;38', 9: '35;37;44;46;50', 10: '51;55', 11: '57', 12: '61', 13: '63', 14: '19;21;49;58;65;66', 15: '56;67;69;71', 16: '72', 17: '74', 18: '59;76', 19: '39;40;47;54;60;70;75;77', 20: '24;29;33;41;45;52;79', 21: '82'}
{0: 0, 2: 1, 4: 2, 11: 3, 5: 4, 9: 4, 13: 4, 15: 4, 23: 5, 12: 6, 14: 6, 31: 6, 1: 7, 3: 7, 17: 7, 18: 7, 30: 7, 32: 7, 34: 8, 38: 8, 35: 9, 37: 9, 44: 9, 46: 9, 50: 9, 51: 10, 55: 10, 57: 11, 61: 12, 63: 13, 19: 14, 21: 14, 49: 14, 58: 14, 65: 14, 66: 14, 56: 15, 67: 15, 69

...

m8 S 0 150
OrderedDict([(0, '0'), (1, '2'), (2, '4'), (3, '11'), (4, '5;9;13;15'), (5, '6;8;10;16;20'), (6, '23'), (7, '12;14;31'), (8, '1;3;17;18;30;32'), (9, '34;38'), (10, '35;37;44;46;50'), (11, '51;55'), (12, '57'), (13, '61'), (14, '63'), (15, '19;21;49;58;65;66'), (16, '56;67;69;71'), (17, '72'), (18, '74'), (19, '59;76'), (20, '39;40;47;54;60;70;75;77'), (21, '24;29;33;41;45;52;79'), (22, '82')])
{0: '0', 1: '2', 2: '11', 3: '5;9;13;15', 4: '6;8;10;16;20', 5: '23', 6: '12;14;31', 7: '1;3;17;18;30;32', 8: '34;38', 9: '35;37;44;46;50', 10: '51;55', 11: '57', 12: '61', 13: '63', 14: '19;21;49;58;65;66', 15: '56;67;69;71', 16: '72', 17: '74', 18: '59;76', 19: '39;40;47;54;60;70;75;77', 20: '24;29;33;41;45;52;79', 21: '82'}
{0: 0, 2: 1, 11: 2, 5: 3, 9: 3, 13: 3, 15: 3, 6: 4, 8: 4, 10: 4, 16: 4, 20: 4, 23: 5, 12: 6, 14: 6, 31: 6, 1: 7, 3: 7, 17: 7, 18: 7, 30: 7, 32: 7, 34: 8, 38: 8, 35: 9, 37: 9, 44: 9, 46: 9, 50: 9, 51: 10, 55: 10, 57: 11, 61: 12, 63: 13, 19: 14, 21: 14, 49: 14, 58:

m8 S 0 159
OrderedDict([(0, '0'), (1, '2'), (2, '4'), (3, '11'), (4, '5;9;13;15'), (5, '6;8;10;16;20'), (6, '23'), (7, '12;14;31'), (8, '1;3;17;18;30;32'), (9, '34;38'), (10, '35;37;44;46;50'), (11, '51;55'), (12, '57'), (13, '61'), (14, '63'), (15, '19;21;49;58;65;66'), (16, '56;67;69;71'), (17, '72'), (18, '74'), (19, '59;76'), (20, '39;40;47;54;60;70;75;77'), (21, '24;29;33;41;45;52;79'), (22, '82')])
{0: '0', 1: '2', 2: '4', 3: '11', 4: '5;9;13;15', 5: '23', 6: '12;14;31', 7: '1;3;17;18;30;32', 8: '34;38', 9: '35;37;44;46;50', 10: '51;55', 11: '57', 12: '61', 13: '63', 14: '19;21;49;58;65;66', 15: '56;67;69;71', 16: '72', 17: '74', 18: '59;76', 19: '39;40;47;54;60;70;75;77', 20: '24;29;33;41;45;52;79', 21: '82'}
{0: 0, 2: 1, 4: 2, 11: 3, 5: 4, 9: 4, 13: 4, 15: 4, 23: 5, 12: 6, 14: 6, 31: 6, 1: 7, 3: 7, 17: 7, 18: 7, 30: 7, 32: 7, 34: 8, 38: 8, 35: 9, 37: 9, 44: 9, 46: 9, 50: 9, 51: 10, 55: 10, 57: 11, 61: 12, 63: 13, 19: 14, 21: 14, 49: 14, 58: 14, 65: 14, 66: 14, 56: 15, 67: 15, 6

m8 S 0 168
OrderedDict([(0, '0'), (1, '2'), (2, '4'), (3, '11'), (4, '5;9;13;15'), (5, '6;8;10;16;20'), (6, '23'), (7, '12;14;31'), (8, '1;3;17;18;30;32'), (9, '34;38'), (10, '35;37;44;46;50'), (11, '51;55'), (12, '57'), (13, '61'), (14, '63'), (15, '19;21;49;58;65;66'), (16, '56;67;69;71'), (17, '72'), (18, '74'), (19, '59;76'), (20, '39;40;47;54;60;70;75;77'), (21, '24;29;33;41;45;52;79'), (22, '82')])
{0: '0', 1: '2', 2: '4', 3: '11', 4: '5;9;13;15', 5: '23', 6: '12;14;31', 7: '1;3;17;18;30;32', 8: '34;38', 9: '35;37;44;46;50', 10: '51;55', 11: '57', 12: '61', 13: '63', 14: '19;21;49;58;65;66', 15: '56;67;69;71', 16: '72', 17: '74', 18: '59;76', 19: '39;40;47;54;60;70;75;77', 20: '24;29;33;41;45;52;79', 21: '82'}
{0: 0, 2: 1, 4: 2, 11: 3, 5: 4, 9: 4, 13: 4, 15: 4, 23: 5, 12: 6, 14: 6, 31: 6, 1: 7, 3: 7, 17: 7, 18: 7, 30: 7, 32: 7, 34: 8, 38: 8, 35: 9, 37: 9, 44: 9, 46: 9, 50: 9, 51: 10, 55: 10, 57: 11, 61: 12, 63: 13, 19: 14, 21: 14, 49: 14, 58: 14, 65: 14, 66: 14, 56: 15, 67: 15, 6

m8 S 0 177
OrderedDict([(0, '0'), (1, '2'), (2, '4'), (3, '11'), (4, '5;9;13;15'), (5, '6;8;10;16;20'), (6, '23'), (7, '12;14;31'), (8, '1;3;17;18;30;32'), (9, '34;38'), (10, '35;37;44;46;50'), (11, '51;55'), (12, '57'), (13, '61'), (14, '63'), (15, '19;21;49;58;65;66'), (16, '56;67;69;71'), (17, '72'), (18, '74'), (19, '59;76'), (20, '39;40;47;54;60;70;75;77'), (21, '24;29;33;41;45;52;79'), (22, '82')])
{0: '0', 1: '2', 2: '4', 3: '11', 4: '5;9;13;15', 5: '23', 6: '12;14;31', 7: '1;3;17;18;30;32', 8: '34;38', 9: '35;37;44;46;50', 10: '51;55', 11: '57', 12: '61', 13: '63', 14: '19;21;49;58;65;66', 15: '56;67;69;71', 16: '72', 17: '74', 18: '59;76', 19: '39;40;47;54;60;70;75;77', 20: '24;29;33;41;45;52;79', 21: '82'}
{0: 0, 2: 1, 4: 2, 11: 3, 5: 4, 9: 4, 13: 4, 15: 4, 23: 5, 12: 6, 14: 6, 31: 6, 1: 7, 3: 7, 17: 7, 18: 7, 30: 7, 32: 7, 34: 8, 38: 8, 35: 9, 37: 9, 44: 9, 46: 9, 50: 9, 51: 10, 55: 10, 57: 11, 61: 12, 63: 13, 19: 14, 21: 14, 49: 14, 58: 14, 65: 14, 66: 14, 56: 15, 67: 15, 6

m8 S 0 186
OrderedDict([(0, '0'), (1, '2'), (2, '4'), (3, '11'), (4, '5;9;13;15'), (5, '6;8;10;16;20'), (6, '23'), (7, '12;14;31'), (8, '1;3;17;18;30;32'), (9, '34;38'), (10, '35;37;44;46;50'), (11, '51;55'), (12, '57'), (13, '61'), (14, '63'), (15, '19;21;49;58;65;66'), (16, '56;67;69;71'), (17, '72'), (18, '74'), (19, '59;76'), (20, '39;40;47;54;60;70;75;77'), (21, '24;29;33;41;45;52;79'), (22, '82')])
{0: '0', 1: '2', 2: '4', 3: '11', 4: '5;9;13;15', 5: '23', 6: '12;14;31', 7: '1;3;17;18;30;32', 8: '34;38', 9: '35;37;44;46;50', 10: '51;55', 11: '57', 12: '61', 13: '63', 14: '19;21;49;58;65;66', 15: '56;67;69;71', 16: '72', 17: '74', 18: '59;76', 19: '39;40;47;54;60;70;75;77', 20: '24;29;33;41;45;52;79', 21: '82'}
{0: 0, 2: 1, 4: 2, 11: 3, 5: 4, 9: 4, 13: 4, 15: 4, 23: 5, 12: 6, 14: 6, 31: 6, 1: 7, 3: 7, 17: 7, 18: 7, 30: 7, 32: 7, 34: 8, 38: 8, 35: 9, 37: 9, 44: 9, 46: 9, 50: 9, 51: 10, 55: 10, 57: 11, 61: 12, 63: 13, 19: 14, 21: 14, 49: 14, 58: 14, 65: 14, 66: 14, 56: 15, 67: 15, 6

In [8]:

# Compare the cluster index to mutation labels that is indicated by the corrected tsvs to the
# one used when producing the adjacency matrix, and confirm it's the same

def get_index_to_cluster_label_from_corrected_sim_tsv(ref_var_fn):
    df = pd.read_csv(ref_var_fn, sep="\t", index_col=False)
    clstr_idx_to_label = {}
    labels = df['character_label'].unique()
    for label in labels:
        idx = int(df[df['character_label']==label]['cluster_index'].unique().item())
        if idx not in clstr_idx_to_label:
            clstr_idx_to_label[idx] = []
        clstr_idx_to_label[idx].append(str(label))
    clstr_idx_to_label = {k:";".join(v) for k,v in clstr_idx_to_label.items()}
    return clstr_idx_to_label
        
for site in sites:

    for mig_type in mig_types:
        site_mig_data_dir = os.path.join(machina_sims_data_dir, site, mig_type)
        seeds = fnmatch.filter(os.listdir(site_mig_data_dir), 'reads_seed*.tsv')
        seeds = [s.replace(".tsv", "").replace("reads_seed", "") for s in seeds]
        for seed in seeds:
            cluster_fn = os.path.join(machina_sims_data_dir, f"{site}_clustered_input", f"cluster_{mig_type}_seed{seed}.txt")

            all_mut_trees_fn = os.path.join(machina_sims_data_dir, f"{site}_mut_trees", f"mut_trees_{mig_type}_seed{seed}.txt")
            trees = fnmatch.filter(os.listdir(os.path.join(machina_sims_data_dir, f"{site}_clustered_input_corrected")), f"cluster_{mig_type}_seed{seed}_tree*.tsv")
            idx_to_cluster_label = dutil.get_idx_to_cluster_label(cluster_fn, ignore_polytomies=True)
            #print(len(df))
#             x = None
            for tree_num in range(len(trees)):
                ref_var_fn = os.path.join(machina_sims_data_dir, f"{site}_clustered_input_corrected", f"cluster_{mig_type}_seed{seed}_tree{tree_num}.tsv")
                corrected_idx_to_cluster_label = get_index_to_cluster_label_from_corrected_sim_tsv(ref_var_fn)
                data = dutil.get_adj_matrices_from_spruce_mutation_trees(all_mut_trees_fn, idx_to_cluster_label, is_sim_data=True)
    
                assert(data[tree_num][1] == corrected_idx_to_cluster_label)
                tree = data[tree_num][0]
                if site=='m8' and mig_type=='mS' and seed=='4' and tree_num==0:
                    print(tree)
                    print(idx_to_cluster_label)
                
                
                

[[0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
OrderedDict([(0, '0'), (1, '1'), (2, '2'), (3, '4;7;8;11;12;17;18'), (4, '19'), (